In [33]:
import pandas as pd

pd.set_option('display.max_columns', None)
df = pd.read_csv("../dataset/Loan_Default.csv")
df.head()


,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,loan_amount,rate_of_interest,Interest_rate_spread,Upfront_charges,term,Neg_ammortization,interest_only,lump_sum_payment,property_value,construction_type,occupancy_type,Secured_by,total_units,income,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
0,24890,2019,cf,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,116500,NaN,NaN,NaN,360.0,not_neg,not_int,not_lpsm,118000.0,sb,pr,home,1U,1740.0,EXP,758,CIB,25-34,to_inst,98.728814,south,direct,1,45.0
1,24891,2019,cf,Male,nopre,type2,p1,l1,nopc,b/c,206500,NaN,NaN,NaN,360.0,not_neg,not_int,lpsm,NaN,sb,pr,home,1U,4980.0,EQUI,552,EXP,55-64,to_inst,NaN,North,direct,1,NaN
2,24892,2019,cf,Male,pre,type1,p1,l1,nopc,nob/c,406500,4.56,0.2000,595.0,360.0,neg_amm,not_int,not_lpsm,508000.0,sb,pr,home,1U,9480.0,EXP,834,CIB,35-44,to_inst,80.019685,south,direct,0,46.0
3,24893,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,456500,4.25,0.6810,NaN,360.0,not_neg,not_int,not_lpsm,658000.0,sb,pr,home,1U,11880.0,EXP,587,CIB,45-54,not_inst,69.376900,North,direct,0,42.0
4,24894,2019,cf,Joint,pre,type1,p1,l1,nopc,nob/c,696500,4.00,0.3042,0.0,360.0,not_neg,not_int,not_lpsm,758000.0,sb,pr,home,1U,10440.0,CRIF,602,EXP,25-34,not_inst,91.886544,North,direct,0,39.0


In [34]:
def calculate_credit_worthiness(
    credit_score, income_usd, dtir1, open_credit, loan_amount_usd, age,
    business_or_commercial, property_value_usd, LTV, term, rate_of_interest,
    neg_ammortization, interest_only, lump_sum_payment
):
    score = 0

    # Credit score
    if credit_score >= 700:
        score += 2
    elif credit_score >= 600:
        score += 1

    # Income (in USD)
    if income_usd > (50000/325):
        score += 2
    elif income_usd > (30000/325):
        score += 1

    # DTI
    if dtir1 < 35:
        score += 2
    elif dtir1 < 45:
        score += 1

    # Open credit lines
    if open_credit <= 3:
        score += 1
    elif open_credit > 7:
        score -= 1

    # Loan amount vs property value (in USD)
    if loan_amount_usd < property_value_usd:
        score += 1

    # Age
    if age >= 30:
        score += 1

    # Business or commercial loan
    if business_or_commercial == "No":
        score += 1

    # LTV
    if LTV < 80:
        score += 1
    elif LTV > 90:
        score -= 1

    # Term
    if term < 240:
        score += 1

    # Rate of interest
    if rate_of_interest < 10:
        score += 1
    elif rate_of_interest > 15:
        score -= 1

    # Loan features
    if neg_ammortization == "No":
        score += 1
    if interest_only == "No":
        score += 1
    if lump_sum_payment == "Yes":
        score += 1

    # Final decision
    if score >= 10:
        return "Good"
    elif score >= 6:
        return "Average"
    else:
        return "Bad"

In [35]:
df=df.drop(columns=['Credit_Worthiness'])

In [36]:
df=df[(df['loan_amount'] < 500000) & (df['term'] < 360)]
df.shape

(25382, 33)

In [37]:
df = df.drop(columns=[
    'business_or_commercial',
    'Neg_ammortization', 'interest_only', 'lump_sum_payment'
])


In [38]:
df

,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,open_credit,loan_amount,rate_of_interest,Interest_rate_spread,Upfront_charges,term,property_value,construction_type,occupancy_type,Secured_by,total_units,income,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
10,24900,2019,cf,Male,nopre,type2,p3,nopc,136500,NaN,NaN,NaN,300.0,168000.0,sb,pr,home,1U,4020.0,EXP,723,CIB,55-64,to_inst,81.250000,North,direct,1,44.0
20,24910,2019,cf,Joint,nopre,type1,p3,nopc,306500,2.990,0.2837,10470.00,180.0,558000.0,sb,pr,home,1U,16860.0,EXP,668,EXP,65-74,to_inst,54.928315,North,direct,0,6.0
38,24928,2019,ncf,Male,pre,type1,p4,nopc,256500,NaN,NaN,NaN,180.0,418000.0,sb,pr,home,1U,3780.0,EXP,780,EXP,65-74,to_inst,61.363636,central,direct,1,45.0
50,24940,2019,cf,Sex Not Available,nopre,type1,p3,nopc,196500,4.125,0.4465,3188.01,312.0,338000.0,sb,pr,home,1U,9060.0,CIB,543,EXP,25-34,to_inst,58.136095,south,direct,0,30.0
51,24941,2019,cf,Male,nopre,type1,p4,nopc,206500,2.990,0.2450,5882.75,180.0,708000.0,sb,pr,home,1U,9600.0,EXP,846,CIB,65-74,to_inst,29.166667,North,direct,0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148652,173542,2019,cf,Male,nopre,type1,p1,nopc,96500,NaN,NaN,NaN,180.0,NaN,sb,ir,home,1U,5460.0,EQUI,608,EXP,55-64,to_inst,NaN,North,direct,1,NaN
148665,173555,2019,cf,Sex Not Available,nopre,type1,p3,nopc,436500,3.125,0.2571,9960.00,180.0,608000.0,sb,pr,home,1U,7860.0,CIB,659,EXP,55-64,to_inst,71.792763,south,direct,0,48.0
148667,173557,2019,cf,Male,nopre,type1,p4,nopc,446500,3.125,0.0816,1226.64,180.0,728000.0,sb,pr,home,1U,6900.0,CIB,702,EXP,45-54,not_inst,61.332418,North,direct,0,49.0
148668,173558,2019,cf,Female,nopre,type1,p4,nopc,196500,3.500,0.5824,4323.33,180.0,278000.0,sb,pr,home,1U,7140.0,EXP,737,EXP,55-64,to_inst,70.683453,North,direct,0,29.0


In [39]:
df=df.drop(columns=['property_value','loan_type','loan_purpose','rate_of_interest','construction_type'])

In [40]:
df

,ID,year,loan_limit,Gender,approv_in_adv,open_credit,loan_amount,Interest_rate_spread,Upfront_charges,term,occupancy_type,Secured_by,total_units,income,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
10,24900,2019,cf,Male,nopre,nopc,136500,NaN,NaN,300.0,pr,home,1U,4020.0,EXP,723,CIB,55-64,to_inst,81.250000,North,direct,1,44.0
20,24910,2019,cf,Joint,nopre,nopc,306500,0.2837,10470.00,180.0,pr,home,1U,16860.0,EXP,668,EXP,65-74,to_inst,54.928315,North,direct,0,6.0
38,24928,2019,ncf,Male,pre,nopc,256500,NaN,NaN,180.0,pr,home,1U,3780.0,EXP,780,EXP,65-74,to_inst,61.363636,central,direct,1,45.0
50,24940,2019,cf,Sex Not Available,nopre,nopc,196500,0.4465,3188.01,312.0,pr,home,1U,9060.0,CIB,543,EXP,25-34,to_inst,58.136095,south,direct,0,30.0
51,24941,2019,cf,Male,nopre,nopc,206500,0.2450,5882.75,180.0,pr,home,1U,9600.0,EXP,846,CIB,65-74,to_inst,29.166667,North,direct,0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148652,173542,2019,cf,Male,nopre,nopc,96500,NaN,NaN,180.0,ir,home,1U,5460.0,EQUI,608,EXP,55-64,to_inst,NaN,North,direct,1,NaN
148665,173555,2019,cf,Sex Not Available,nopre,nopc,436500,0.2571,9960.00,180.0,pr,home,1U,7860.0,CIB,659,EXP,55-64,to_inst,71.792763,south,direct,0,48.0
148667,173557,2019,cf,Male,nopre,nopc,446500,0.0816,1226.64,180.0,pr,home,1U,6900.0,CIB,702,EXP,45-54,not_inst,61.332418,North,direct,0,49.0
148668,173558,2019,cf,Female,nopre,nopc,196500,0.5824,4323.33,180.0,pr,home,1U,7140.0,EXP,737,EXP,55-64,to_inst,70.683453,North,direct,0,29.0


In [41]:
df['occupancy_type'].value_counts()

occupancy_type
pr    23638
ir     1132
sr      612
Name: count, dtype: int64

In [42]:
df=df.drop(columns=['occupancy_type'])

In [43]:
df

,ID,year,loan_limit,Gender,approv_in_adv,open_credit,loan_amount,Interest_rate_spread,Upfront_charges,term,Secured_by,total_units,income,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
10,24900,2019,cf,Male,nopre,nopc,136500,NaN,NaN,300.0,home,1U,4020.0,EXP,723,CIB,55-64,to_inst,81.250000,North,direct,1,44.0
20,24910,2019,cf,Joint,nopre,nopc,306500,0.2837,10470.00,180.0,home,1U,16860.0,EXP,668,EXP,65-74,to_inst,54.928315,North,direct,0,6.0
38,24928,2019,ncf,Male,pre,nopc,256500,NaN,NaN,180.0,home,1U,3780.0,EXP,780,EXP,65-74,to_inst,61.363636,central,direct,1,45.0
50,24940,2019,cf,Sex Not Available,nopre,nopc,196500,0.4465,3188.01,312.0,home,1U,9060.0,CIB,543,EXP,25-34,to_inst,58.136095,south,direct,0,30.0
51,24941,2019,cf,Male,nopre,nopc,206500,0.2450,5882.75,180.0,home,1U,9600.0,EXP,846,CIB,65-74,to_inst,29.166667,North,direct,0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148652,173542,2019,cf,Male,nopre,nopc,96500,NaN,NaN,180.0,home,1U,5460.0,EQUI,608,EXP,55-64,to_inst,NaN,North,direct,1,NaN
148665,173555,2019,cf,Sex Not Available,nopre,nopc,436500,0.2571,9960.00,180.0,home,1U,7860.0,CIB,659,EXP,55-64,to_inst,71.792763,south,direct,0,48.0
148667,173557,2019,cf,Male,nopre,nopc,446500,0.0816,1226.64,180.0,home,1U,6900.0,CIB,702,EXP,45-54,not_inst,61.332418,North,direct,0,49.0
148668,173558,2019,cf,Female,nopre,nopc,196500,0.5824,4323.33,180.0,home,1U,7140.0,EXP,737,EXP,55-64,to_inst,70.683453,North,direct,0,29.0


In [44]:
df['Secured_by'].value_counts()

Secured_by
home    25376
land        6
Name: count, dtype: int64

In [45]:
df=df.drop(columns=['Secured_by'])

In [46]:
df

,ID,year,loan_limit,Gender,approv_in_adv,open_credit,loan_amount,Interest_rate_spread,Upfront_charges,term,total_units,income,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
10,24900,2019,cf,Male,nopre,nopc,136500,NaN,NaN,300.0,1U,4020.0,EXP,723,CIB,55-64,to_inst,81.250000,North,direct,1,44.0
20,24910,2019,cf,Joint,nopre,nopc,306500,0.2837,10470.00,180.0,1U,16860.0,EXP,668,EXP,65-74,to_inst,54.928315,North,direct,0,6.0
38,24928,2019,ncf,Male,pre,nopc,256500,NaN,NaN,180.0,1U,3780.0,EXP,780,EXP,65-74,to_inst,61.363636,central,direct,1,45.0
50,24940,2019,cf,Sex Not Available,nopre,nopc,196500,0.4465,3188.01,312.0,1U,9060.0,CIB,543,EXP,25-34,to_inst,58.136095,south,direct,0,30.0
51,24941,2019,cf,Male,nopre,nopc,206500,0.2450,5882.75,180.0,1U,9600.0,EXP,846,CIB,65-74,to_inst,29.166667,North,direct,0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148652,173542,2019,cf,Male,nopre,nopc,96500,NaN,NaN,180.0,1U,5460.0,EQUI,608,EXP,55-64,to_inst,NaN,North,direct,1,NaN
148665,173555,2019,cf,Sex Not Available,nopre,nopc,436500,0.2571,9960.00,180.0,1U,7860.0,CIB,659,EXP,55-64,to_inst,71.792763,south,direct,0,48.0
148667,173557,2019,cf,Male,nopre,nopc,446500,0.0816,1226.64,180.0,1U,6900.0,CIB,702,EXP,45-54,not_inst,61.332418,North,direct,0,49.0
148668,173558,2019,cf,Female,nopre,nopc,196500,0.5824,4323.33,180.0,1U,7140.0,EXP,737,EXP,55-64,to_inst,70.683453,North,direct,0,29.0


In [47]:
df['total_units'].value_counts()

total_units
1U    25197
2U      130
3U       38
4U       17
Name: count, dtype: int64

In [48]:
df['credit_type'].value_counts()

credit_type
CIB     8319
CRIF    7351
EXP     6794
EQUI    2918
Name: count, dtype: int64

In [49]:
df=df.drop(columns=['Security_Type','Region'])

In [50]:
df

,ID,year,loan_limit,Gender,approv_in_adv,open_credit,loan_amount,Interest_rate_spread,Upfront_charges,term,total_units,income,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Status,dtir1
10,24900,2019,cf,Male,nopre,nopc,136500,NaN,NaN,300.0,1U,4020.0,EXP,723,CIB,55-64,to_inst,81.250000,1,44.0
20,24910,2019,cf,Joint,nopre,nopc,306500,0.2837,10470.00,180.0,1U,16860.0,EXP,668,EXP,65-74,to_inst,54.928315,0,6.0
38,24928,2019,ncf,Male,pre,nopc,256500,NaN,NaN,180.0,1U,3780.0,EXP,780,EXP,65-74,to_inst,61.363636,1,45.0
50,24940,2019,cf,Sex Not Available,nopre,nopc,196500,0.4465,3188.01,312.0,1U,9060.0,CIB,543,EXP,25-34,to_inst,58.136095,0,30.0
51,24941,2019,cf,Male,nopre,nopc,206500,0.2450,5882.75,180.0,1U,9600.0,EXP,846,CIB,65-74,to_inst,29.166667,0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148652,173542,2019,cf,Male,nopre,nopc,96500,NaN,NaN,180.0,1U,5460.0,EQUI,608,EXP,55-64,to_inst,NaN,1,NaN
148665,173555,2019,cf,Sex Not Available,nopre,nopc,436500,0.2571,9960.00,180.0,1U,7860.0,CIB,659,EXP,55-64,to_inst,71.792763,0,48.0
148667,173557,2019,cf,Male,nopre,nopc,446500,0.0816,1226.64,180.0,1U,6900.0,CIB,702,EXP,45-54,not_inst,61.332418,0,49.0
148668,173558,2019,cf,Female,nopre,nopc,196500,0.5824,4323.33,180.0,1U,7140.0,EXP,737,EXP,55-64,to_inst,70.683453,0,29.0


In [51]:
df['total_monthly_payment']=(df['dtir1']*df['income'])/100
df['DTI'] = (df['total_monthly_payment']/df['income'])*100

In [52]:
y = df['Status']
X = df[['Credit_Score','Gender', 'income', 'open_credit', 'age','term','loan_amount','DTI']]
X.columns

Index(['Credit_Score', 'Gender', 'income', 'open_credit', 'age', 'term',
       'loan_amount', 'DTI'],
      dtype='object')

In [53]:
X = X.fillna(X.median(numeric_only=True))
X = pd.get_dummies(X, drop_first=True)
X.isna().sum()

Credit_Score                0
income                      0
term                        0
loan_amount                 0
DTI                         0
Gender_Joint                0
Gender_Male                 0
Gender_Sex Not Available    0
open_credit_opc             0
age_35-44                   0
age_45-54                   0
age_55-64                   0
age_65-74                   0
age_<25                     0
age_>74                     0
dtype: int64

In [54]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [55]:
# %pip install imbalanced-learn

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, Y_resampled = ros.fit_resample(X_train, y_train)

In [56]:
Y_resampled.value_counts()

Status
0    14974
1    14974
Name: count, dtype: int64

In [57]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,   # limit tree depth
    random_state=42
)


# Train
rf.fit(X_resampled, Y_resampled)


RandomForestClassifier(max_depth=5, random_state=42)

In [58]:
y_pred= rf.predict(X_test)
y_pred

array([1, 0, 1, ..., 0, 0, 0])

In [59]:
rf.score(X_test, y_test)

0.6698837896395509

In [60]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, target_names=[str(c) for c in rf.classes_]))

              precision    recall  f1-score   support

           0       0.88      0.64      0.74      3744
           1       0.43      0.75      0.55      1333

    accuracy                           0.67      5077
   macro avg       0.65      0.70      0.64      5077
weighted avg       0.76      0.67      0.69      5077



In [61]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def evaluate(model, X_test, y_test):
    preds = model.predict(X_test)
    return {
        "Accuracy": accuracy_score(y_test, preds),
        "Precision": precision_score(y_test, preds),
        "Recall": recall_score(y_test, preds),
        "F1": f1_score(y_test, preds),
        "ROC-AUC": roc_auc_score(y_test, preds)
    }

print("Random Forest:", evaluate(rf, X_test, y_test))


Random Forest: {'Accuracy': 0.6698837896395509, 'Precision': 0.42711432214194645, 'Recall': 0.7539384846211553, 'F1': 0.5453065653825284, 'ROC-AUC': 0.6969478747892102}


In [62]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(rf, X, y, cv=5, scoring="accuracy")
print("Cross-validation Accuracy:", scores.mean())


Cross-validation Accuracy: 0.7534470882064744


In [63]:
import joblib
joblib.dump(rf, 'personal_loan_default_model.pkl')

['personal_loan_default_model.pkl']

In [64]:
import joblib
joblib.dump(list(X.columns), 'personal_X_columns.pkl')

['personal_X_columns.pkl']